In [1]:
pip install termcolor

Note: you may need to restart the kernel to use updated packages.


# Matrix trace

- Define Word Lists: We already have the words provided. We'll use these to generate the puzzle.

- Choose Grid Size: Determine the grid size based on the maximum word length. We'll ensure the grid size is at least 5 and proportional to the word length.

- Create Empty Grid: Initialize an empty grid of the determined size.

- Place Words: Randomly place words onto the grid while ensuring they fit and do not overlap.

- Fill Grid: Fill the remaining empty spaces on the grid with random letters.

- Print Grid: Display the grid with words highlighted.

In [2]:
import tkinter as tk
import random
import string
import ipywidgets as widgets
from IPython.display import display
import tkinter as tk
import random
import string
import tkinter as tk
import random
import string
import ipywidgets as widgets
from IPython.display import display
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import tkinter as tk
import random
import string
import json
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [3]:
import json
import random
import string
import tkinter as tk
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os

# Function to load words from JSON files
def load_words_from_json(file_path):
    with open(file_path, 'r') as f:
        words = json.load(f)
    return words

# Load words from JSON files
animals_words = load_words_from_json('./JSON/animals.json')
foods_words = load_words_from_json('./JSON/foods.json')
colors_words = load_words_from_json('./JSON/colors.json')
# Add more categories as needed

print(animals_words)

# Combine loaded words into a dictionary
common_words = {
    'Animals': animals_words,
    'Foods': foods_words,
    'Colors': colors_words,
    # Add more categories as needed
}



{'PreK': {'Animals': ['CAT', 'DOG', 'COW', 'FISH', 'PIG', 'FOX', 'BAT', 'RAT', 'OWL', 'ANT', 'BEE', 'FLY', 'ELK', 'EMU', 'GNU', 'EEL', 'YAK', 'APE']}, 'Kindergarten': {'Animals': ['TAHR', 'ROOK', 'MOLE', 'MINK', 'PONY', 'SLUG', 'BASS', 'TIGER', 'DUCK', 'SWAN', 'TOAD', 'LION', 'GULL', 'SOLE', 'DEER', 'SHEEP', 'BIRD', 'KIWI', 'PUMA', 'TERN', 'TICK', 'TUNA', 'PANDA', 'CROW', 'FOWL', 'CARP', 'FROG', 'GOAT', 'BOAR', 'KOALA', 'CRAB', 'KITE', 'NEWT', 'MOTH', 'WOLF', 'PIKE', 'CLAM', 'LOON', 'MULE', 'HAWK', 'LARK', 'LYNX', 'ORCA', 'WORM', 'FISH', 'HARE', 'GUAN', 'FLEA', 'HORSE', 'WASP', 'ZEBRA', 'BEAR', 'MITE', 'VOLE', 'CLAM']}, '1st Grade': {'Animals': ['RABBIT', 'TURTLE', 'MONKEY', 'KANGAROO', 'ELEPHANT', 'GIRAFFE', 'HIPPO', 'RHINO', 'CHEETAH', 'LEOPARD', 'GORILLA', 'FLAMINGO', 'CAMELEON', 'LADYBUG']}}


In [4]:
# Function to get words by category
def get_words_by_category(category):
    return common_words.get(category, {})

def get_words_by_grade_and_type(grade, word_type):
    category_words = get_words_by_category(word_type)
    return category_words.get(grade, {}).get(word_type, [])

def create_empty_grid(size):
    return [['' for _ in range(size)] for _ in range(size)]

def place_word_in_grid(grid, word, start_row, start_col, direction):
    if direction == 'horizontal':
        for i, char in enumerate(word):
            grid[start_row][start_col + i] = char
    elif direction == 'vertical':
        for i, char in enumerate(word):
            grid[start_row + i][start_col] = char
    elif direction == 'diagonal':
        for i, char in enumerate(word):
            grid[start_row + i][start_col + i] = char

def can_place_word(grid, word, start_row, start_col, direction, size):
    if direction == 'horizontal' and start_col + len(word) > size:
        return False
    if direction == 'vertical' and start_row + len(word) > size:
        return False
    if direction == 'diagonal' and (start_row + len(word) > size or start_col + len(word) > size):
        return False
    for i in range(len(word)):
        if direction == 'horizontal' and grid[start_row][start_col + i] not in ['', word[i]]:
            return False
        if direction == 'vertical' and grid[start_row + i][start_col] not in ['', word[i]]:
            return False
        if direction == 'diagonal' and grid[start_row + i][start_col + i] not in ['', word[i]]:
            return False
    return True

def fill_grid_randomly(grid, size):
    for row in range(size):
        for col in range(size):
            if grid[row][col] == '':
                grid[row][col] = random.choice(string.ascii_uppercase)

def generate_word_search(words, num_words, size=10):
    grid = create_empty_grid(size)
    directions = ['horizontal', 'vertical', 'diagonal']
    word_positions = []

    words_to_use = random.sample(words, num_words)

    for word in words_to_use:
        placed = False
        while not placed:
            direction = random.choice(directions)
            start_row = random.randint(0, size-1)
            start_col = random.randint(0, size-1)
            if can_place_word(grid, word, start_row, start_col, direction, size):
                place_word_in_grid(grid, word, start_row, start_col, direction)
                word_positions.append((word, start_row, start_col, direction))
                placed = True
    
    fill_grid_randomly(grid, size)
    return grid, word_positions

def create_word_search_window(grid, word_positions, size, add_solution_to_pdf, filename):
    root = tk.Tk()
    root.title("Word Search Grid")

    colors = ["#%02x%02x%02x" % (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(len(word_positions))]

    for row in range(size):
        for col in range(size):
            text = grid[row][col]
            color = "#FFFFFF"  # Default background color
            for idx, pos in enumerate(word_positions):
                word, start_row, start_col, direction = pos
                if direction == 'horizontal' and row == start_row and start_col <= col < start_col + len(word):
                    color = colors[idx]
                elif direction == 'vertical' and col == start_col and start_row <= row < start_row + len(word):
                    color = colors[idx]
                elif direction == 'diagonal' and (row - start_row) == (col - start_col) and start_row <= row < start_row + len(word):
                    color = colors[idx]
            cell = tk.Label(root, text=text, bg=color, width=4, height=2, font=('Helvetica', 12, 'bold'))
            cell.grid(row=row, column=col, padx=1, pady=1)

    if add_solution_to_pdf:
        # Generate PDF with solution grid
        create_solution_pdf(grid, filename, size)
        print(f"Solution PDF created: {filename}")

    root.mainloop()

def create_solution_pdf(grid, filename, size):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    cell_size = width / size

    c.setFont("Helvetica", 12)

    for row in range(size):
        for col in range(size):
            x = col * cell_size
            y = height - (row + 1) * cell_size
            c.drawString(x + cell_size / 2 - 5, y + cell_size / 2 - 5, grid[row][col])

    c.save()

def on_button_clicked(name, grid_size, grade, word_type, num_words, print_pdf, filename):
    words = get_words_by_grade_and_type(grade, word_type)
    if not words:
        print("No words found for the selected grade and type.")
        return

    add_solution_to_pdf = print_pdf
    grid, word_positions = generate_word_search(words, num_words, grid_size)
    
    # Define the folder path based on word type
    folder_path = f'./results/{word_type.lower()}/'
    os.makedirs(folder_path, exist_ok=True)

    # Define the full filename
    full_filename = os.path.join(folder_path, f"{name}_word_search_{word_type.lower()}{grid_size}_{num_words}.pdf")

    create_word_search_window(grid, word_positions, grid_size, add_solution_to_pdf, full_filename)



In [5]:
# Example usage
on_button_clicked("x5", 10, "PreK", "Foods", 8, True, "word_search.pdf")

Solution PDF created: ./results/foods/x5_word_search_foods10_8.pdf
